# Part I. ETL Pipeline for Pre-Processing the Files

## PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# Checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# Join the file path and roots with the subdirectories using glob
    # Did not work: file_path_list = glob.glob(os.path.join(root,'*'))
    # Hard code the file path because above original code threw an error.
    file_path_list = glob.glob('/home/workspace/event_data/*.csv')

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# Initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
# extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# Check the number of rows in your csv file
# there should be 6,821 rows
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of the project.

## Now the CSV file titled <font color=red>event_datafile_new.csv</font> is ready to query. It is located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# Make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Create tables to run the following queries. Remember, with Apache Cassandra the database tables are modeled on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
# Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

drop_query = "DROP TABLE IF EXISTS session_library"
try:
    session.execute(drop_query)
    print("Table dropped.")
except Exception as e:
    print(e)

table_query = "CREATE TABLE IF NOT EXISTS session_library "
table_query = table_query + "(sessionId int, itemInSession int, artist text, song text, length float,  PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(table_query)
    print("Table created.")
except Exception as e:
    print(e)

# Column indices for reference:
# 0 artist
# 1 firstName
# 2 gender
# 3 itemInSession
# 4 lastName
# 5 length
# 6 level
# 7 location
# 8 sessionId
# 9 song
# 10 userId

# Learning note:
# The brackets in Cassandra play a important role, if you wish to define PRIMARY KEY((a,b),c,d)
# this declaration would typically mean that you want to define your partitioning key as a,b : 
## these are typically your "where" in CQL and c and d are clustering key, typically maintaining sorting order.

Table dropped.
Table created.


In [10]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_library (sessionId, itemInSession, artist, song, length)"
        query = query +  "VALUES (%s, %s, %s, %s, %s)"
        # Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
# Add in the SELECT statement to verify the data was entered into the table
query = "SELECT * FROM session_library WHERE sessionId = 338 and itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.sessionid, row.iteminsession, row.artist, row.song, row.length)

338 4 Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [12]:
# Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
drop_query = "DROP TABLE IF EXISTS user_library"
try:
    session.execute(drop_query)
    print("Table dropped.")
except Exception as e:
    print(e)

table_query = "CREATE TABLE IF NOT EXISTS user_library "
table_query = table_query + "(userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, PRIMARY KEY ((userId, sessionId), itemInSession ))"
try:
    session.execute(table_query)
    print("Table created.")
except Exception as e:
    print(e)        

Table dropped.
Table created.


In [13]:
file = 'event_datafile_new.csv'

# Column indices for reference:
# 0 artist
# 1 firstName
# 2 gender
# 3 itemInSession
# 4 lastName
# 5 length
# 6 level
# 7 location
# 8 sessionId
# 9 song
# 10 userId

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO user_library (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query +  "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [14]:
## Add in the SELECT statement to verify the data was entered into the table
# Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query = "SELECT * FROM user_library WHERE userid = 10 and sessionId = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.userid, row.sessionid, row.iteminsession, row.artist, row.song, row.firstname, row.lastname)

10 182 0 Down To The Bone Keep On Keepin' On Sylvie Cruz
10 182 1 Three Drives Greece 2000 Sylvie Cruz
10 182 2 Sebastien Tellier Kilometer Sylvie Cruz
10 182 3 Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [15]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
drop_query = "DROP TABLE IF EXISTS song_library"
try:
    session.execute(drop_query)
    print("Table dropped.")
except Exception as e:
    print(e)

table_query = "CREATE TABLE IF NOT EXISTS song_library "
table_query = table_query + "(song text, firstName text, lastName text, userId int, PRIMARY KEY ((song), firstName, lastName ))"
try:
    session.execute(table_query)
    print("Table created.")
except Exception as e:
    print(e)     

                    

Table dropped.
Table created.


In [16]:
file = 'event_datafile_new.csv'

# Column indices for reference:
# 0 artist
# 1 firstName
# 2 gender
# 3 itemInSession
# 4 lastName
# 5 length
# 6 level
# 7 location
# 8 sessionId
# 9 song
# 10 userId

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_library (song, firstName, lastName, userId)"
        query = query +  "VALUES (%s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[9], line[1], line[4], int(line[10])))

In [17]:
# Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = "SELECT * FROM song_library WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.song, row.firstname, row.lastname, row.userid)

All Hands Against His Own Jacqueline Lynch 29
All Hands Against His Own Sara Johnson 95
All Hands Against His Own Tegan Levine 80


### Drop the tables before closing out the sessions

In [18]:
## Drop the table before closing out the sessions

drop_query = "DROP TABLE IF EXISTS session_library"
try:
    session.execute(drop_query)
    print("Table dropped.")
except Exception as e:
    print(e)
    
drop_query = "DROP TABLE IF EXISTS user_library"
try:
    session.execute(drop_query)
    print("Table dropped.")
except Exception as e:
    print(e)

drop_query = "DROP TABLE IF EXISTS song_library"
try:
    session.execute(drop_query)
    print("Table dropped.")
except Exception as e:
    print(e)

Table dropped.
Table dropped.
Table dropped.


### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()